<a href="https://colab.research.google.com/github/AnshMittal1811/BERT/blob/master/Keyphrase_Extraction_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment for Rapidken.AI

In [1]:
!ls

sample_data


# Task 2
Using the model trained and built using Pranav's code, please extract key phrases the text file.

### Trying SciBERT according to the Instructions in BERT-keyphrase-extraction Repository

In [2]:
!git clone https://github.com/pranav-ust/BERT-keyphrase-extraction

Cloning into 'BERT-keyphrase-extraction'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.


In [3]:
!pip install pytorch-pretrained-BERT

     |████████████████████████████████| 133kB 4.7MB/s 


In [4]:
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar

--2020-06-06 20:59:10--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.237.48
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.237.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410593280 (392M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 391.57M  51.7MB/s    in 7.6s    

2020-06-06 20:59:18 (51.3 MB/s) - ‘scibert_scivocab_uncased.tar’ saved [410593280/410593280]



In [5]:
!ls

BERT-keyphrase-extraction  sample_data	scibert_scivocab_uncased.tar


In [0]:
!mkdir BERT-keyphrase-extraction/model

In [7]:
!tar -C BERT-keyphrase-extraction/model -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [0]:
import os
os.chdir('BERT-keyphrase-extraction/model/scibert_scivocab_uncased')

In [9]:
!ls

vocab.txt  weights.tar.gz


In [10]:
!tar -C ./ -xvf weights.tar.gz

bert_config.json
pytorch_model.bin


In [11]:
!ls

bert_config.json  pytorch_model.bin  vocab.txt	weights.tar.gz


In [0]:
!mv pytorch_model.bin ../
!mv vocab.txt ../
!mv bert_config.json ../

In [0]:
os.chdir('../../')

In [14]:
!ls

data		evaluate.py  metrics.py  README.md  utils.py
data_loader.py	experiments  model	 train.py


In [0]:
!rm -r model/scibert_scivocab_uncased

Here, we change the `params.json` file in `experiments/base_model` to increase number of epochs to 50 along with `min_epoch_num : 25` and `patience_num : 10`. 

In [0]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
Starting training for 50 epoch(s)
Epoch 1/50
/usr/loca

In [17]:
!python evaluate.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model --restore_file best

Loading the dataset...
loading vocabulary file model/vocab.txt
- done.
Starting evaluation...
- Test metrics: loss: 00.37; f1: 46.06
             precision    recall  f1-score   support

          I      36.35     62.87     46.06       921

avg / total      36.35     62.87     46.06       921



In [0]:
!rm -r model

In [19]:
!ls

data		evaluate.py  metrics.py   README.md  utils.py
data_loader.py	experiments  __pycache__  train.py


In [0]:
os.chdir('./experiments/base_model')

In [0]:
!rm ./best.pth.tar
!rm ./evaluate.log
!rm ./last.pth.tar
!rm ./train.log

# Task 1 

Use the code in
https://github.com/pranav-ust/BERT-keyphrase-extraction to train it
with default parameters and evaluate the model for just Subtask 1 with
the following change: Instead of using SciBERT, please use BERT. As
you can see, Pranav has given evaluation results with SciBERT as F1
score: 0.6259; Precision: 0.5986; Recall: 0.6558; Support: 921. Please
give the results using BERT instead.

### Trying BERT according using BERT-keyphrase-extraction Repository

In [0]:
os.chdir('../../')
!mkdir ./model

In [26]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

--2020-06-06 18:39:11--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.167.128, 2a00:1450:400c:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.167.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G   179MB/s    in 7.1s    

2020-06-06 18:39:18 (168 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]



In [27]:
!unzip uncased_L-24_H-1024_A-16.zip -d ./model

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: ./model/uncased_L-24_H-1024_A-16/
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: ./model/uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_config.json  


In [0]:
!rm -r uncased_L-24_H-1024_A-16.zip

In [0]:
os.chdir('./model/uncased_L-24_H-1024_A-16')

In [0]:
!mv bert_model.ckpt.meta ../
!mv bert_model.ckpt.data-00000-of-00001 ../
!mv bert_config.json ../
!mv vocab.txt ../
!mv bert_model.ckpt.index ../

In [0]:
!rm -r ../uncased_L-24_H-1024_A-16

In [0]:
os.chdir('/content/BERT-keyphrase-extraction/model')

In [34]:
!ls

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


#### Converting ckpt files to pytorch_model.bin

In [35]:
!pip install transformers

In [0]:
from transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert
import torch

In [37]:
config = BertConfig.from_json_file('./bert_config.json')
print("Building PyTorch model from configuration: {}".format(str(config)))
model = BertForPreTraining(config)

# Load weights from tf checkpoint
load_tf_weights_in_bert(model, config, './bert_model.ckpt')

# Save pytorch-model
print("Save PyTorch model to {}".format('./pytorch_model.bin'))
torch.save(model.state_dict(), './pytorch_model.bin')

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Save PyTorch model to ./pytorch_model.bin


In [0]:
!rm bert_model.ckpt.data-00000-of-00001
!rm bert_model.ckpt.index
!rm bert_model.ckpt.meta

In [0]:
os.chdir('../')

In [40]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
Traceback (most recent call last):
  File "train.py", line 168, in <module>
    train_data = data_loader.load_data('train')
  File "/content/BERT-keyphrase-extraction/data_loader.py", line 83, in load_data
    self.load_sentences_tags(sentences_file, tags_path, data)
  File "/content/BERT-keyphrase-extraction/data_loader.py", line 51, in load_sentences_tags
    sentences.append(self.tokenizer.convert_tokens_to_ids(tokens))
  File "/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/tokenization.py", line 121, in convert_tokens_to_ids
    ids.append(self.vocab[token])
KeyError: 'nonzero'


There is a difference in the vocabulary from the SciBERT models so, we use it instead for this dataset.

In [41]:
os.chdir('../')
!tar -C BERT-keyphrase-extraction/model -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [43]:
os.chdir('BERT-keyphrase-extraction/model')
!rm vocab.txt
!mv ./scibert_scivocab_uncased/vocab.txt ./

rm: cannot remove 'scibert_scivocab_uncased': Is a directory


In [0]:
!rm -r scibert_scivocab_uncased

In [46]:
os.chdir('../')
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
Starting training for

Now, we need to refer to use the recommended values of batch size of 4 and sequence length of 512, with 6 epochs, if GPU's VRAM is around 11 GB. (changed in experiments/base_model/params.json)
But, even after reducing that we still get CUDA out of Memory error so we use batch size of 2 and sequence length of 512, with 50 epochs (`min_epoch_num:20` and `patience_num:10`). Learning rate = 3e-4 and weight decay = 0.01

In [58]:
!python evaluate.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model --restore_file best

Loading the dataset...
loading vocabulary file model/vocab.txt
- done.
Starting evaluation...
- Test metrics: loss: 00.61; f1: 04.03
             precision    recall  f1-score   support

          I       3.41      4.91      4.03       916

avg / total       3.41      4.91      4.03       916

